In [ ]:
import types
import urllib3, requests, json
import pandas as pd

df_data_1 = pd.read_csv('bases 2024/leads_fixed.csv', low_memory=False, sep=',', encoding='utf8')
df_data_1.shape

In [ ]:
df_data_1 = df_data_1.drop(columns=['BINARY_LIFECYCLE'], axis=1)
df_data_1.head()

In [ ]:
df_fields = df_data_1.to_json(orient='table', force_ascii=False, index=False)
df_values = df_data_1.to_json(orient='values', force_ascii=False)
j_fields = json.loads(df_fields)
j_values = json.loads(df_values)
fields = []
values = []

for index1, value in enumerate(j_values):
        values.append(value)

for f in j_fields['schema']['fields']:
    fields.append(f['name'])

In [ ]:
# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = "IBM_WATSON_API_KEY"
token_response = requests.post('https://iam.ng.bluemix.net/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/<DEPLOYMENT_ID>/predictions?version=2023-03-07', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
response_json = json.loads(response_scoring.text)

In [ ]:
response_json['predictions'][0]

In [ ]:
result = json.loads(response_scoring.text)

result_data = {'contact ID': df_data_1['RECORD_ID'], 'Negative chance': None, 'Positive chance': None, 'Conversion': None}
result_df = pd.DataFrame(data=result_data)

for i in range(len(result['predictions'][0]['values'])):
    result_df.at[i, 'Negative chance'] = "{:.2%}".format(result['predictions'][0]['values'][i][1][0])
    result_df.at[i, 'Positive chance'] = "{:.2%}".format(result['predictions'][0]['values'][i][1][1])
    result_df.at[i, 'Conversion'] = "Yes" if (result['predictions'][0]['values'][i][0]) else "No"

In [ ]:
export = result_df.to_excel(r'scorings 2023/' + 'lead' + '_scoring.xlsx', index=False, encoding='utf-8')